<a href="https://colab.research.google.com/github/anuva04/AgroBuddy/blob/main/Copy_of_crops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification of crops
### Random forest classifier and XGBoost

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
dataset = pd.read_csv('datasets.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
print(X_train)

[[7.2  0.46 0.81 0.24]
 [6.11 0.5  0.55 0.37]
 [7.83 0.42 0.63 0.2 ]
 ...
 [5.18 0.58 0.75 0.13]
 [6.54 0.58 0.21 0.16]
 [5.74 0.44 0.55 0.64]]


In [ ]:
print(X_test)

[[5.3  0.87 0.55 0.04]
 [5.59 0.52 0.7  0.26]
 [5.53 0.85 0.53 0.11]
 ...
 [5.12 0.86 0.44 0.02]
 [6.23 0.75 0.39 0.19]
 [7.24 0.76 0.79 0.29]]


In [ ]:
print(y_train)

['tomato' 'orange' 'lily' ... 'coffee' 'paddy' 'cauliflower']


In [ ]:
print(y_test)

['blackberry' 'potato' 'marigold' ... 'blackberry' 'gladiola' 'rose']


### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
print(X_train)

[[ 1.30286324 -1.2198511   1.23261443  0.0348833 ]
 [-0.23470409 -0.96876251 -0.53810632  0.87568361]
 [ 2.19154894 -1.47093969  0.00673083 -0.22382449]
 ...
 [-1.54657347 -0.46658533  0.82398657 -0.67656312]
 [ 0.37185917 -0.46658533 -2.85366423 -0.48253228]
 [-0.75663062 -1.3453954  -0.53810632  2.62196119]]


In [ ]:
print(X_test)

[[-1.3773      1.35380696 -0.53810632 -1.25865565]
 [-0.96822245 -0.84321821  0.48346334  0.16423719]
 [-1.05285919  1.22826266 -0.67431561 -0.80591702]
 ...
 [-1.6312102   1.29103481 -1.28725741 -1.38800954]
 [-0.06543063  0.60054118 -1.62778063 -0.28850144]
 [ 1.35928773  0.66331333  1.09640514  0.35826803]]


### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators = 13, criterion = 'entropy', random_state = 0)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=13, random_state=0)

In [ ]:
print(rf_classifier.predict(sc.transform([[7.37, 0.86, 0.8, 0.26]])))

['rose']


In [ ]:
y_pred_rf = rf_classifier.predict(X_test)
print(np.concatenate((y_pred_rf.reshape(len(y_pred_rf),1), y_test.reshape(len(y_test),1)),1))

[['blackberry' 'blackberry']
 ['potato' 'potato']
 ['marigold' 'marigold']
 ...
 ['blackberry' 'blackberry']
 ['gladiola' 'gladiola']
 ['rose' 'rose']]


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred_rf)
print(cm)
accuracy_score(y_test, y_pred_rf)

[[120   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0
    0   0   0]
 [  0  98  19   0   0   0   6   0   3   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0  12 123   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0
    0   0   0]
 [  0   0   0 132   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0   0   0   0 246   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0   0   0   0   0  86   0   0   0   0   0   0   0   0   0   0   0   0
    0  39   0]
 [  0   3   0   0   0   0 223   0  16   0   0   0   2   0   0   0   0   2
    0   0   0]
 [  0   0   0   0   0   0   0 258   0   0   0   0   3   0   0   0   0   0
    0   0   0]
 [  0   0   0   0   0   0  38   0  88   0   0   0   0   0   0   0   0   5
    0   0   0]
 [  0   0   0   0   0   0   0   0   0  42   0  42   0   0  29   0   0  11
    0   0   0]
 [  0   0   0   0   0   0   0   0   0   0 238   0   0   0   0   0   0   0
    0   0   0]
 [  0   0   0   0   0

0.8655719454494851

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = rf_classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 85.91 %
Standard Deviation: 0.98 %


### XGBoost

In [ ]:
from xgboost import XGBClassifier
params = {
    'colsample_bytree': 0.7,
    'learning_rate': 0.01,
    'max_depth': 6,
    'n_estimators': 500
}
xgbclassifer = XGBClassifier(params=params)
xgbclassifer.fit(X_train, y_train)

XGBClassifier(objective='multi:softprob',
              params={'colsample_bytree': 0.7, 'learning_rate': 0.01,
                      'max_depth': 6, 'n_estimators': 500})

### Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}
grid_search = GridSearchCV(estimator = xgbclassifer,
                           param_grid = params,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

Best Accuracy: 86.80 %
Best Parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 500}


In [ ]:
y_pred_xgb = xgbclassifer.predict(X_test)
cm = confusion_matrix(y_test, y_pred_xgb)
print(cm)
print('Accuracy:')
accuracy_score(y_test, y_pred_xgb)

[[120   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0 104  15   0   0   0   7   0   0   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0  18 118   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0   0   0 132   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0   0   0   0 246   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0   0   0   0   0  96   0   0   0   0   0   0   0   0   0   0   0   0
    0  29   0]
 [  0   0   0   0   0   0 234   0  10   0   0   0   1   0   0   0   0   1
    0   0   0]
 [  0   0   0   0   0   0   0 261   0   0   0   0   0   0   0   0   0   0
    0   0   0]
 [  0   2   0   0   0   0  38   0  90   0   0   0   0   0   0   0   0   1
    0   0   0]
 [  0   0   0   0   0   0   0   0   0  21   0  63   0   0  40   0   0   0
    0   0   0]
 [  0   0   0   0   0   0   0   0   0   0 238   0   0   0   0   0   0   0
    0   0   0]
 [  0   0   0   0   0

0.8792095741720011

In [ ]:
accuracies = cross_val_score(estimator = xgbclassifer, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 87.29 %
Standard Deviation: 0.92 %


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test, y_pred_xgb, average='micro')

(0.8792095741720011, 0.8792095741720011, 0.879209574172001, None)

In [ ]:
precision_recall_fscore_support(y_test, y_pred_xgb, average='macro')

(0.896993804429526, 0.8636753242816592, 0.858530983773338, None)

In [ ]:
precision_recall_fscore_support(y_test, y_pred_xgb, average='weighted')

(0.9010051852079464, 0.8792095741720011, 0.8715633265595669, None)